## <b>Data Cleaning Notebook:</b> [IEEE-CIS Fraud Detection](https://www.kaggle.com/c/ieee-fraud-detection)

### Table of Contents
##### <b>Part 1: Introduction</b>
- Brief description of the notebook's purpose and the datasets involved.
##### <b>Part 2: Setup and Imports</b>
- Import necessary libraries and functions.
##### <b>Part 3: Data Loading</b>
- Details about the data sources and their reliability.
- Read in the data files.
##### <b>Part 4: Initial Data Inspection</b>
- Initial data inspection and understanding.
- Check and print data types of each column.
- Determine the shape (number of rows and columns) of the data.
##### <b>Part 5: Data Reshaping and Combining</b>
- Standardize column names for consistency.
- Reshape data to fit analytical needs.
- Combine multiple datasets if necessary.
- Set appropriate index if necessary.
##### <b>Part 6: Data Cleaning</b>
- Identify and remove duplicate rows.
- Identify and handle missing data.
- Impute or drop missing values as appropriate.
- Any additional cleaning steps specific to the data.
##### <b>Part 7: Verifying Data Quality</b>
- Verify the quality of the cleaned data.
- Handle and log any errors during the data cleaning process.
##### <b>Part 8: Save the Analytical Set</b>
- Save the cleaned and prepared dataset for analysis.
Part 9: Outcome and Summary
- Summarize the outcomes of the data cleaning process.

### <b>Introduction</b>
##### <u>Purpose of the Notebook</u>
This notebook aims to clean and preprocess the IEEE-CIS Fraud Detection dataset in preparation for building a model to predict the probability that an online transaction is fraudulent, as indicated by the binary target variable `isFraud`. The steps involved include importing necessary libraries, loading the data, performing initial inspections, cleaning and reshaping the data, and verifying the data quality. The final cleaned dataset will be saved for further analysis and model building.
##### <u>Dataset Description</u>
The dataset is used to predict the likelihood of fraudulent online transactions. It is divided into two main files: `identity` and `transaction`, which are linked by the `TransactionID` field. Not all transactions have corresponding identity information.
##### Transaction Data (train_transaction.csv and test_transaction.csv)
- `ProductCD`: Product code.
- `card1` - `card6`: Payment card information.
- `addr1`, `addr2`: Address information.
- `P_emaildomain`, `R_emaildomain`: Payer and recipient email domains.
- `M1` - `M9`: Miscellaneous features.
- `TransactionDT`: A timedelta from a given reference datetime (not an actual timestamp).
##### Identity Data (train_identity.csv and test_identity.csv)
- `DeviceType`: Type of device used for the transaction.
- `DeviceInfo`: Information about the device.
- `id_12` - `id_38`: Various identity-related features.
##### Additional Files
sample_submission.csv: A sample submission file in the correct format for submitting predictions.

### <b>Setup and Imports</b>

In [107]:
# Import necessary libraries
import pandas as pd
import os
import sys

# Add the src directory to the system path
src_path = os.path.abspath('../src/')
sys.path.append(src_path)

# Import custom data cleaning functions
from data_cleaning import standardize_col_names, null_counts

##### <b>The data cleaning functions in the src file are:</b>

##### `standardize_col_names(df)`
- Standardizes column names of a dataframe. It will remove white space, replace spaces with underscores, and eliminate special characters (including parenthesis and slashes). All letters are converted to lowercase.
- Returns a copy of the dataframe.

##### `null_counts(df)`
- Returns a dataframe containing the number of null values in each column of a given dataframe.

### <b>Data Loading</b>

##### <u>Details about the Data Sources and Their Reliability</u>
The dataset for the IEEE-CIS Fraud Detection competition was provided by Vesta Corporation, a leading company in guaranteed e-commerce payment solutions. Founded in 1995, Vesta pioneered fully guaranteed card-not-present (CNP) payment transactions for the telecommunications industry. Over the years, Vesta has significantly expanded its data science and machine learning capabilities, establishing itself as a global leader in guaranteed e-commerce payments.

Vesta's reputation for handling and guaranteeing over $18 billion in transactions annually underscores the reliability and robustness of the provided dataset. This reliability is crucial for building accurate and effective fraud detection models.

In [108]:
# Define the location of the input data
input_loc = '../data/raw/'

# Read in the data
try:
    sample_submission = pd.read_csv(os.path.join(input_loc, 'sample_submission.csv'))
    test_identity = pd.read_csv(os.path.join(input_loc, 'test_identity.csv'), dtype={'column_name': str})  # Specify dtype for columns with mixed types
    test_transaction = pd.read_csv(os.path.join(input_loc, 'test_transaction.csv'))
    train_identity = pd.read_csv(os.path.join(input_loc, 'train_identity.csv'), dtype={'column_name': str})  # Specify dtype for columns with mixed types
    train_transaction = pd.read_csv(os.path.join(input_loc, 'train_transaction.csv'))
except Exception as e:
    print(f"Error reading data: {e}")

# Print the head of the dataframes to verify the content
print(sample_submission.head())
print(test_identity.head())
print(test_transaction.head())
print(train_identity.head())
print(train_transaction.head())

/var/folders/sr/c4hqwd5d201c391w690mwpxc0000gn/T/ipykernel_52806/989143938.py:7: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  test_identity = pd.read_csv(os.path.join(input_loc, 'test_identity.csv'), dtype={'column_name': str})  # Specify dtype for columns with mixed types
/var/folders/sr/c4hqwd5d201c391w690mwpxc0000gn/T/ipykernel_52806/989143938.py:9: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  train_identity = pd.read_csv(os.path.join(input_loc, 'train_identity.csv'), dtype={'column_name': str})  # Specify dtype for columns with mixed types


   TransactionID  isFraud
0        3663549      0.5
1        3663550      0.5
2        3663551      0.5
3        3663552      0.5
4        3663553      0.5
   TransactionID variable value
0        3663586    id_01 -45.0
1        3663588    id_01   0.0
2        3663597    id_01  -5.0
3        3663601    id_01 -45.0
4        3663602    id_01 -95.0
   TransactionID  TransactionDT  TransactionAmt ProductCD  card1  card2  \
0        3663549       18403224           31.95         W  10409  111.0   
1        3663550       18403263           49.00         W   4272  111.0   
2        3663551       18403310          171.00         W   4476  574.0   
3        3663552       18403310          284.95         W  10989  360.0   
4        3663553       18403317           67.95         W  18018  452.0   

   card3       card4  card5  card6  ...  V331  V332  V333  V334 V335 V336  \
0  150.0        visa  226.0  debit  ...   NaN   NaN   NaN   NaN  NaN  NaN   
1  150.0        visa  226.0  debit  ...   NaN  

<b>Data Loading and Initial Look:</b> The `DtypeWarning` indicates that certain columns have mixed data types. This can cause performance issues or unexpected behavior during analysis.  

### <b>Initial Data Inspection</b>

<u>What's in the data?</u>

In [109]:
sample_submission.head()

,TransactionID,isFraud
0,3663549,0.5
1,3663550,0.5
2,3663551,0.5
3,3663552,0.5
4,3663553,0.5


In [110]:
train_identity.head()

,TransactionID,variable,value
0,2987004,id_01,0.0
1,2987008,id_01,-5.0
2,2987010,id_01,-5.0
3,2987011,id_01,-5.0
4,2987016,id_01,0.0


Observation: Identity information is stored in a long format, with each `transactionid` having multiple `variable` and `value` pairs.

In [111]:
train_transaction.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V331,V332,V333,V334,V335,V336,V337,V338,V339,V340
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [112]:
test_identity.head()

,TransactionID,variable,value
0,3663586,id_01,-45.0
1,3663588,id_01,0.0
2,3663597,id_01,-5.0
3,3663601,id_01,-45.0
4,3663602,id_01,-95.0


Observation: Follows the same long format as `train_identity`.

In [113]:
test_transaction.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V331,V332,V333,V334,V335,V336,V337,V338,V339,V340
0,3663549,18403224,31.95,W,10409,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3663550,18403263,49.00,W,4272,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3663551,18403310,171.00,W,4476,574.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3663552,18403310,284.95,W,10989,360.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3663553,18403317,67.95,W,18018,452.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Summary:
- The data includes both transaction and identity information for training and testing.
- The `train_transaction` and `test_transaction` datasets are wide, containing many features.
- The `train_identity` and `test_identity` datasets are long, containing key-value pairs for additional identity attributes.
- The `sample_submission` dataset provides a template for predictions, indicating that `isFraud` in this context might represent a probability.

<u>What are the data types of each column?</u>

In [114]:
# Print the data types of each dataset
print("Data types in sample_submission:\n", sample_submission.dtypes, "\n")
print("Data types in test_identity:\n", test_identity.dtypes, "\n")
print("Data types in test_transaction:\n", test_transaction.dtypes, "\n")
print("Data types in train_identity:\n", train_identity.dtypes, "\n")
print("Data types in train_transaction:\n", train_transaction.dtypes, "\n")

Data types in sample_submission:
 TransactionID      int64
isFraud          float64
dtype: object 

Data types in test_identity:
 TransactionID     int64
variable         object
value            object
dtype: object 

Data types in test_transaction:
 TransactionID       int64
TransactionDT       int64
TransactionAmt    float64
ProductCD          object
card1               int64
card2             float64
card3             float64
card4              object
card5             float64
card6              object
addr1             float64
addr2             float64
dist1             float64
dist2             float64
P_emaildomain      object
R_emaildomain      object
C1                float64
C2                float64
C3                float64
C4                float64
C5                float64
C6                float64
C7                float64
C8                float64
C9                float64
C10               float64
C11               float64
C12               float64
C13               flo

Summary:
- `sample_submission`: `TransactionID` is an integer, and `isFraud` is a float (indicating a probability).
- `test_identity` and `train_identity`: `TransactionID` is an integer, variable is an object, and value is an object (mixed types will need to be resolved).
- `test_transaction` and `train_transaction`: Mostly numerical with a few categorical variables like `ProductCD` and `card4`.

<u>What is the shape of the data?</u>

In [117]:
# Print the shape of each dataset
print("Shape of sample_submission:", sample_submission.shape)
print("Shape of test_identity:", test_identity.shape)
print("Shape of test_transaction:", test_transaction.shape)
print("Shape of train_identity:", train_identity.shape)
print("Shape of train_transaction:", train_transaction.shape)

Shape of sample_submission: (506691, 2)
Shape of test_identity: (5676280, 3)
Shape of test_transaction: (506691, 402)
Shape of train_identity: (5769320, 3)
Shape of train_transaction: (590540, 403)


Summary:
- `sample_submission` has 506,691 rows and 2 columns. This is the template for our predictions.
- `test_identity` and `train_identity` are long-format datasets with approximately 5.7 million rows and 3 columns each.
- `test_transaction` and `train_transaction` are wide-format datasets with around 500,000 rows and over 400 columns each. The training set includes the target variable `isFraud`.

### <b>Data Reshaping and Combining</b>

##### <u>Standardize Column Names:</u> 
Applying the function `standardize_col_names` to ensure all DataFrame column names follow a consistent format(lowercase, no spaces or special characters). 

In [118]:
# Standardize column names for consistency
sample_submission = standardize_col_names(sample_submission)
test_identity = standardize_col_names(test_identity)
test_transaction = standardize_col_names(test_transaction)
train_identity = standardize_col_names(train_identity)
train_transaction = standardize_col_names(train_transaction)

<u>Reshape the Identity Data</u>

In [119]:
# Reshape the identity data to wide format
train_identity_wide = train_identity.pivot(index='transactionid', columns='variable', values='value')
test_identity_wide = test_identity.pivot(index='transactionid', columns='variable', values='value')

# Reset the index to ensure transactionid is a column
train_identity_wide.reset_index(inplace=True)
test_identity_wide.reset_index(inplace=True)

<u>Merge the Reshaped Identity Data with Transaction Data</u>

In [120]:
# Merge identity and transaction datasets for training and test sets
train_merged = pd.merge(train_transaction, train_identity_wide, on='transactionid', how='left')
test_merged = pd.merge(test_transaction, test_identity_wide, on='transactionid', how='left')

##### <u>Set the Index:</u>
Let's proceed with setting the appropriate index for each DataFrame. Typically, `transactionid` is a good candidate for setting as the index since it uniquely identifies each transaction.

In [121]:
# Set the index for each DataFrame
sample_submission.set_index('transactionid', inplace=True)
test_identity.set_index('transactionid', inplace=True)
test_transaction.set_index('transactionid', inplace=True)
train_identity.set_index('transactionid', inplace=True)
train_transaction.set_index('transactionid', inplace=True)

<u>Verify the Merges</u>

In [122]:
# Verify the shape of the merged datasets
print("Shape of train_merged:", train_merged.shape)
print("Shape of test_merged:", test_merged.shape)

# Check the first few rows of the merged datasets to ensure they look correct
print("First few rows of train_merged:\n", train_merged.head())
print("First few rows of test_merged:\n", test_merged.head())

# Check for any null values in key columns to ensure the merge was successful
print("Null values in train_merged:\n", train_merged.isnull().sum())
print("Null values in test_merged:\n", test_merged.isnull().sum())

Shape of train_merged: (590540, 443)
Shape of test_merged: (506691, 442)
First few rows of train_merged:
    transactionid  isfraud  transactiondt  transactionamt productcd  card1  \
0        2987000        0          86400            68.5         W  13926   
1        2987001        0          86401            29.0         W   2755   
2        2987002        0          86469            59.0         W   4663   
3        2987003        0          86499            50.0         W  18132   
4        2987004        0          86506            50.0         H   4497   

   card2  card3       card4  card5  ...     id_29        id_30  \
0    NaN  150.0    discover  142.0  ...       NaN          NaN   
1  404.0  150.0  mastercard  102.0  ...       NaN          NaN   
2  490.0  150.0        visa  166.0  ...       NaN          NaN   
3  567.0  150.0  mastercard  117.0  ...       NaN          NaN   
4  514.0  150.0  mastercard  102.0  ...  NotFound  Android 7.0   

                 id_31  id_32     

### <b>Data Cleaning</b>
- Identify and remove duplicate rows.
- Identify and handle missing data.
- Impute or drop missing values as appropriate.
- Any additional cleaning steps specific to the data.

##### <u>Remove Duplicate Rows:</u>
We'll identify and remove any duplicate rows in the datasets.

In [123]:
# Remove duplicate rows after merging
train_merged = train_merged.drop_duplicates()
test_merged = test_merged.drop_duplicates()

Sanity check!

In [124]:
# Verify the shape of the datasets after removing duplicates
print("Shape of train_merged after removing duplicates:", train_merged.shape)
print("Shape of test_merged after removing duplicates:", test_merged.shape)

Shape of train_merged after removing duplicates: (590540, 443)
Shape of test_merged after removing duplicates: (506691, 442)


<u>Identify Missing Data</u>

In [125]:
# Check for missing data in train_merged
missing_train = train_merged.isnull().sum()
missing_train = missing_train[missing_train > 0]
missing_train_percentage = (missing_train / len(train_merged)) * 100

# Check for missing data in test_merged
missing_test = test_merged.isnull().sum()
missing_test = missing_test[missing_test > 0]
missing_test_percentage = (missing_test / len(test_merged)) * 100

# Display the missing data statistics
missing_data_train = pd.DataFrame({'Missing Values': missing_train, 'Percentage': missing_train_percentage})
missing_data_test = pd.DataFrame({'Missing Values': missing_test, 'Percentage': missing_test_percentage})

print("Missing Data in train_merged:\n", missing_data_train)
print("\nMissing Data in test_merged:\n", missing_data_test)

Missing Data in train_merged:
                Missing Values  Percentage
card2                    8933    1.512683
card3                    1565    0.265012
card4                    1577    0.267044
card5                    4259    0.721204
card6                    1571    0.266028
addr1                   65706   11.126427
addr2                   65706   11.126427
dist1                  352271   59.652352
dist2                  552913   93.628374
p_emaildomain           94456   15.994852
r_emaildomain          453249   76.751617
d1                       1269    0.214888
d2                     280797   47.549192
d3                     262878   44.514851
d4                     168922   28.604667
d5                     309841   52.467403
d6                     517353   87.606767
d7                     551623   93.409930
d8                     515614   87.312290
d9                     515614   87.312290
d10                     76022   12.873302
d11                    279287   47.293494
d12

##### <u>Drop Columns with High Percentage of Missing Values</u>
Let's drop columns with more than 50% missing values.

In [126]:
# Drop columns with more than 50% missing values
threshold = 50  # Percentage

# Filter columns to drop based on the threshold
cols_to_drop_train = train_merged.columns[train_merged.isnull().mean() * 100 > threshold]
cols_to_drop_test = test_merged.columns[test_merged.isnull().mean() * 100 > threshold]

train_merged = train_merged.drop(columns=cols_to_drop_train)
test_merged = test_merged.drop(columns=cols_to_drop_test)

# Verify the shape after dropping columns
print("Shape of train_merged after dropping high-missing columns:", train_merged.shape)
print("Shape of test_merged after dropping high-missing columns:", test_merged.shape)

Shape of train_merged after dropping high-missing columns: (590540, 225)
Shape of test_merged after dropping high-missing columns: (506691, 228)


<u>Impute Missing Values for Specified Categorical Columns</u>

In [127]:
# List of categorical columns to impute with mode
categorical_columns = [
    'productCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2',
    'p_emaildomain', 'r_emaildomain', 'm1', 'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9',
    'devicetype', 'deviceinfo', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18',
    'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28',
    'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38'
]

# Ensure that only existing columns in train_merged and test_merged are included
categorical_columns_train = [col for col in categorical_columns if col in train_merged.columns]
categorical_columns_test = [col for col in categorical_columns if col in test_merged.columns]

# Impute missing values with mode
train_merged[categorical_columns_train] = train_merged[categorical_columns_train].apply(lambda x: x.fillna(x.mode()[0]))
test_merged[categorical_columns_test] = test_merged[categorical_columns_test].apply(lambda x: x.fillna(x.mode()[0]))

# Verify that there are no missing values left in the specified categorical columns
remaining_missing_train = train_merged[categorical_columns_train].isnull().sum().sum()
remaining_missing_test = test_merged[categorical_columns_test].isnull().sum().sum()

print("Remaining missing values in specified categorical columns (train):", remaining_missing_train)
print("Remaining missing values in specified categorical columns (test):", remaining_missing_test)

Remaining missing values in specified categorical columns (train): 0
Remaining missing values in specified categorical columns (test): 0


Sanity check! 

In [128]:
# Verify the shape of the datasets after handling missing data
print("Shape of train_merged after handling missing data:", train_merged.shape)
print("Shape of test_merged after handling missing data:", test_merged.shape)

# Check if there are any remaining missing values in the entire dataset
remaining_missing_train = train_merged.isnull().sum().sum()
remaining_missing_test = test_merged.isnull().sum().sum()

print("Remaining missing values in train_merged:", remaining_missing_train)
print("Remaining missing values in test_merged:", remaining_missing_test)

Shape of train_merged after handling missing data: (590540, 225)
Shape of test_merged after handling missing data: (506691, 228)
Remaining missing values in train_merged: 13054413
Remaining missing values in test_merged: 5429534


<u>Identify Remaining Missing Values</u> 

In [129]:
# Function to list columns with missing values
def list_columns_with_missing_values(df):
    missing_count = df.isnull().sum()
    missing_columns = missing_count[missing_count > 0]
    return missing_columns

# List columns with missing values in train_merged
missing_columns_train = list_columns_with_missing_values(train_merged)
print("Columns with missing values in train_merged:\n", missing_columns_train)

# List columns with missing values in test_merged
missing_columns_test = list_columns_with_missing_values(test_merged)
print("\nColumns with missing values in test_merged:\n", missing_columns_test)

Columns with missing values in train_merged:
 d1        1269
d2      280797
d3      262878
d4      168922
d10      76022
d11     279287
d15      89113
v1      279287
v2      279287
v3      279287
v4      279287
v5      279287
v5      279287
v6      279287
v7      279287
v8      279287
v9      279287
v10     279287
v11     279287
v12      76073
v12      76073
v13      76073
v14      76073
v15      76073
v16      76073
v17      76073
v18      76073
v19      76073
v20      76073
v21      76073
v22      76073
v23      76073
v24      76073
v25      76073
v26      76073
v27      76073
v28      76073
v29      76073
v30      76073
v31      76073
v32      76073
v33      76073
v34      76073
v35     168969
v36     168969
v37     168969
v38     168969
v39     168969
v40     168969
v41     168969
v42     168969
v42     168969
v43     168969
v44     168969
v45     168969
v46     168969
v47     168969
v48     168969
v49     168969
v50     168969
v51     168969
v52     168969
v53      77096
v54      

<u>Impute Missing Numerical Values Using the Mean</u>

In [136]:
# List of numerical columns to impute with mean
numerical_columns = [
    'v1', 'v2', 'v3', 'v4','v5', 'v6', 'v7', 'v8', 'v9', 'v10', 'v11', 'v12', 'v13', 'v14', 'v15', 'v16', 'v17', 'v18',
    'v19', 'v20', 'v21', 'v22', 'v23', 'v24', 'v25', 'v26', 'v27', 'v28', 'v29', 'v30', 'v31', 
    'v32', 'v33', 'v34', 'v35', 'v36', 'v37', 'v38', 'v39', 'v40', 'v41', 'v42', 'v43', 'v44', 
    'v45', 'v46', 'v47', 'v48', 'v49', 'v50', 'v51', 'v52', 'v53', 'v54', 'v55', 'v56', 'v57', 
    'v58', 'v59', 'v60', 'v61', 'v62', 'v63', 'v64', 'v65', 'v66', 'v67', 'v68', 'v69', 'v70', 
    'v71', 'v72', 'v73', 'v74', 'v75', 'v76', 'v77', 'v78', 'v79', 'v80', 'v81', 'v82', 'v83', 
    'v84', 'v85', 'v86', 'v87', 'v88', 'v89', 'v90', 'v91', 'v92', 'v93', 'v94', 'v95', 'v96', 
    'v97', 'v98', 'v99', 'v100', 'v101', 'v102', 'v103', 'v104', 'v105', 'v106', 'v107', 'v108', 
    'v109', 'v110', 'v111', 'v112', 'v113', 'v114', 'v115', 'v116', 'v117', 'v118', 'v119', 'v120', 
    'v121', 'v122', 'v123', 'v124', 'v125', 'v126', 'v127', 'v128', 'v129', 'v130', 'v131', 'v132', 
    'v133', 'v134', 'v135', 'v136', 'v137', 'v279', 'v280', 'v281', 'v282', 'v283', 'v284', 'v285', 
    'v286', 'v287', 'v288', 'v289', 'v290', 'v291', 'v292', 'v293', 'v294', 'v295', 'v296', 'v297', 
    'v298', 'v299', 'v300', 'v301', 'v302', 'v303', 'v304', 'v305', 'v306', 'v307', 'v308', 'v309', 
    'v310', 'v311', 'v312', 'v313', 'v314', 'v315', 'v316', 'v317', 'v318', 'v319', 'v320', 'v321',
    'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13', 'c14', 'd1',
    'd2', 'd3', 'd4', 'd5', 'd10', 'd11', 'd15'
]

# Ensure that only existing columns in train_merged and test_merged are included
numerical_columns_train = [col for col in numerical_columns if col in train_merged.columns]
numerical_columns_test = [col for col in numerical_columns if col in test_merged.columns]

# Impute missing values with mean for train_merged
for col in numerical_columns_train:
    train_merged[col] = train_merged[col].fillna(train_merged[col].mean())

# Impute missing values with mean for test_merged
for col in numerical_columns_test:
    test_merged[col] = test_merged[col].fillna(test_merged[col].mean())

# Verify that there are no missing values left in the specified numerical columns
remaining_missing_train = train_merged[numerical_columns_train].isnull().sum().sum()
remaining_missing_test = test_merged[numerical_columns_test].isnull().sum().sum()

print("Remaining missing values in specified numerical columns (train):", remaining_missing_train)
print("Remaining missing values in specified numerical columns (test):", remaining_missing_test)

Remaining missing values in specified numerical columns (train): 0
Remaining missing values in specified numerical columns (test): 0


Sanity Check!

In [138]:
# Check for missing data in train_merged
missing_train = train_merged.isnull().sum()
missing_train = missing_train[missing_train > 0]
missing_train_percentage = (missing_train / len(train_merged)) * 100

# Check for missing data in test_merged
missing_test = test_merged.isnull().sum()
missing_test = missing_test[missing_test > 0]
missing_test_percentage = (missing_test / len(test_merged)) * 100

# Display the missing data statistics
missing_data_train = pd.DataFrame({'Missing Values': missing_train, 'Percentage': missing_train_percentage})
missing_data_test = pd.DataFrame({'Missing Values': missing_test, 'Percentage': missing_test_percentage})

print("Missing Data in train_merged:\n", missing_data_train)
print("\nMissing Data in test_merged:\n", missing_data_test)

Missing Data in train_merged:
 Empty DataFrame
Columns: [Missing Values, Percentage]
Index: []

Missing Data in test_merged:
 Empty DataFrame
Columns: [Missing Values, Percentage]
Index: []


### <b>Verifying Data Quality</b>

In [143]:
# Function to log and print any errors encountered
def log_error(message):
    with open("data_cleaning_errors.log", "a") as log_file:
        log_file.write(message + "\n")
    print(message)

# Function to verify data quality
def verify_data_quality(df, df_name):
    try:
        # Check for remaining missing values
        missing_values = df.isnull().sum().sum()
        if missing_values > 0:
            log_error(f"Error: {df_name} contains {missing_values} missing values.")
        else:
            print(f"{df_name} contains no missing values.")
        
        # Check for duplicate rows
        duplicate_rows = df.duplicated().sum()
        if duplicate_rows > 0:
            log_error(f"Error: {df_name} contains {duplicate_rows} duplicate rows.")
        else:
            print(f"{df_name} contains no duplicate rows.")
        
        # Check data types
        print(f"Data types in {df_name}:\n", df.dtypes)
    
    except Exception as e:
        log_error(f"Error verifying data quality for {df_name}: {str(e)}")

# Verify the quality of the cleaned data
verify_data_quality(train_merged, "train_merged")
verify_data_quality(test_merged, "test_merged")

train_merged contains no missing values.
train_merged contains no duplicate rows.
Data types in train_merged:
 transactionid       int64
isfraud             int64
transactiondt       int64
transactionamt    float64
productcd          object
card1               int64
card2             float64
card3             float64
card4              object
card5             float64
card6              object
addr1             float64
addr2             float64
p_emaildomain      object
c1                float64
c2                float64
c3                float64
c4                float64
c5                float64
c6                float64
c7                float64
c8                float64
c9                float64
c10               float64
c11               float64
c12               float64
c13               float64
c14               float64
d1                float64
d2                float64
d3                float64
d4                float64
d10               float64
d11               float64
d15  

### <b>Save the Analytical Set</b>

In [140]:
# Save the cleaned DataFrames
train_merged.to_csv('jc_train_data_cleaned.csv', index=False)
test_merged.to_csv('jc_test_data_cleaned.csv', index=False)

print("Cleaned data has been saved.")

Cleaned data has been saved.


### <b>Outcome and Summary</b>

In [144]:
def summarize_data_cleaning():
    print("Data Cleaning Process Summary:")
    
    # Initial shapes (replace with actual initial shapes)
    initial_train_shape = (590540, 443)
    initial_test_shape = (506691, 442)
    
    # Final shapes
    final_train_shape = train_merged.shape
    final_test_shape = test_merged.shape
    
    print(f"Initial shape of train_merged: {initial_train_shape}")
    print(f"Initial shape of test_merged: {initial_test_shape}")
    
    print(f"Final shape of train_merged: {final_train_shape}")
    print(f"Final shape of test_merged: {final_test_shape}")
    
    # Remaining missing values
    remaining_missing_train = train_merged.isnull().sum().sum()
    remaining_missing_test = test_merged.isnull().sum().sum()
    
    print(f"Remaining missing values in train_merged: {remaining_missing_train}")
    print(f"Remaining missing values in test_merged: {remaining_missing_test}")
    
    # Log file for errors
    try:
        with open("data_cleaning_errors.log", "r") as log_file:
            errors = log_file.read()
        if errors:
            print("Errors encountered during data cleaning:")
            print(errors)
        else:
            print("No errors encountered during data cleaning.")
    except FileNotFoundError:
        print("No errors encountered during data cleaning.")
    
    print("Data cleaning process completed successfully.")

# Summarize the outcomes of the data cleaning process
summarize_data_cleaning()

Data Cleaning Process Summary:
Initial shape of train_merged: (590540, 443)
Initial shape of test_merged: (506691, 442)
Final shape of train_merged: (590540, 225)
Final shape of test_merged: (506691, 228)
Remaining missing values in train_merged: 0
Remaining missing values in test_merged: 0
Errors encountered during data cleaning:
Error: test_merged contains 706072 missing values.

Data cleaning process completed successfully.
